In [1]:
#Use Python 3.5
import sys
print(sys.version)

3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]


In [2]:
import os
import pickle
import warnings
warnings.filterwarnings("ignore")
import gensim
import numpy as np
import re
from nltk.tokenize import word_tokenize
from gensim.models.word2vec import Word2Vec
from inspect import getsourcefile
from os.path import abspath,relpath

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical
from keras.layers.wrappers import TimeDistributed

Using TensorFlow backend.


In [4]:
current_path=abspath(getsourcefile(lambda:0)).replace(relpath(getsourcefile(lambda:0)),"")

In [5]:
# for x in tgAi_w2v_temp_model.most_similar(u"elisabeth"):
#     print(x[0])

In [6]:
def pickle_open(file_name):
    with open(file_name, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        return u.load()

In [7]:
Word2Vector_models_path=os.path.join(current_path,"Word2Vector_models")
fname="tgi_w2v_temp_model_python_3.w2v"
tgi_w2v_temp_model = Word2Vec.load(os.path.join(Word2Vector_models_path,fname))

In [8]:
#Data comes from Training_set_labeling_clean.ipynb
training_pickle_path=os.path.join(current_path,"Training_Pickle")
training_data=pickle_open(os.path.join(training_pickle_path,"tgi_training_data_pickle_2.p"))

In [9]:
def label_data(string,sub_strings,sub_strings_label,w2v_model=tgi_w2v_temp_model,word2vec_length=150):
    #String is a Unicode
    #Function comes from Training_set_labeling_clean.ipynb
    #Take the string and identify the boundaries where judge names lie
    divisions=[0,len(string)]
    for sub_string in sub_strings:
        if (sub_string!=0) and (len(re.findall(sub_string,string))!=0):
            for m in re.finditer(sub_string,string):
                divisions.append(m.start())
                divisions.append(m.end())
    divisions=sorted(divisions)

    div_string=[]
    for index in range(len(divisions)-1):
        div_string.append(string[divisions[index]:divisions[index+1]])

    div_string_label=[]
    for sub_string,sub_string_label in zip(sub_strings,sub_strings_label):
        sub_string=str(sub_string)
        #print(type(sub_string))
        div_string_label.append(sub_string_label*np.array(np.asarray(div_string)==np.asarray(sub_string)))

    div_string_label=np.sum(div_string_label,0)

    tokenized_string=[]
    tokenized_string_label=[]
    for index in range(len(div_string)):
        temp=word_tokenize(div_string[index])
        tokenized_string+=temp
        try:
            tokenized_string_label+=[int(div_string_label[index])]*len(temp)
        except:
            tokenized_string_label+=[int(div_string_label)]*len(temp)
    tokens_lower=[word.lower() for word in tokenized_string]
    mul=2
    
    tokens_shape_features=[[mul*int(str(word).isnumeric()) if int(str(word).isnumeric()) else mul*-1,mul*int(str(word).isalpha()) if int(str(word).isalpha()) else mul*-1,mul*int(str(word).islower()) if int(str(word).islower()) else mul*-1,mul*int(str(word).isupper()) if int(str(word).isupper()) else mul*-1] for word in tokenized_string]
    tokens_word2vec_features=[]
    
    for word in tokens_lower:
        try:
            tokens_word2vec_features.append(list(w2v_model[word]))
        except:
            tokens_word2vec_features.append(list(np.random.rand(word2vec_length,1)))
    for index in range(len(tokenized_string)):
        if tokenized_string[index]=="0.0":
            tokenized_string_label[index]=0
    return tokenized_string,tokenized_string_label,np.hstack((tokens_word2vec_features,tokens_shape_features)).tolist()

In [10]:
def token_to_file_features(tokenized_string,tokenized_string_label,token_features,sent_len=20):
    file_features=[]
    for temp in [tokenized_string,tokenized_string_label,token_features]:
        sent_boundaries=list(np.arange(0,len(temp),sent_len))+[len(temp)]
        sent_features=[]
        for x in range(len(sent_boundaries)-1):
            sent_features.append(temp[sent_boundaries[x]:sent_boundaries[x+1]])
        file_features.append(np.array(sent_features).tolist())
        
    return file_features

In [11]:
all_file_tokens=[]
all_file_features=[]
all_file_labels=[]

for num in range(0,len(training_data)):
    #print(str(num),end=",")
    string=training_data.loc[num]["text_data"]
    sub_strings=[training_data.loc[num]["judge_name_1"],training_data.loc[num]["judge_name_2"],training_data.loc[num]["judge_name_3"]]
    sub_strings_label=[1,2,3]
    
    tokenized_string,tokenized_string_label,token_features=label_data(string,sub_strings,sub_strings_label)
    single_tokenized_string,single_tokenized_string_label,single_token_features=token_to_file_features(tokenized_string,tokenized_string_label,token_features)
    all_file_tokens.append(single_tokenized_string)
    all_file_features.append(single_token_features)
    all_file_labels.append(single_tokenized_string_label)
    #print(" ".join(np.hstack(single_tokenized_string)[np.hstack(single_tokenized_string_label)!=0])+"\n")



In [12]:
X_train=[sequence.pad_sequences(all_file_features[i],maxlen=20,dtype='float32',padding='post',truncating='post') for i in range(100)]
X_train_padded=np.vstack(X_train)

X_train_tokens=[sequence.pad_sequences(all_file_tokens[i],maxlen=20,dtype='str',padding='post',truncating='post') for i in range(100)]
X_train_tokens_padded=np.vstack(X_train_tokens)

In [13]:
all_file_labels_array=np.asarray(all_file_labels)
for j in range(len((all_file_labels_array))):
    temp=[]
    for i in range(len(all_file_labels_array[j])):
        temp.append(to_categorical(all_file_labels_array[j][i],len(np.unique(sub_strings_label))+1))
    all_file_labels_array[j]=temp

In [14]:
y_train=[sequence.pad_sequences(all_file_labels_array[i],maxlen=20,dtype='float32',padding='post',truncating='post') for i in range(100)]
y_train_padded=np.vstack(y_train)

In [15]:
temp=X_train_tokens_padded[y_train_padded[:,:,0]!=1]

temp1=X_train_tokens_padded[temp!="0.0"]
temp2=y_train_padded[temp!="0.0"]
temp3=X_train_padded[temp!="0.0"]

print(temp1.shape,temp2.shape,temp3.shape)

(520, 20) (520, 20, 4) (520, 20, 154)


In [95]:
rep=50
temp=[temp1 for n in range(rep)]
X_train_tokens_padded_new=np.vstack((X_train_tokens_padded,np.array(np.vstack(temp))))

temp=[temp2 for n in range(rep)]
y_train_padded_new=np.vstack((y_train_padded,np.array(np.vstack(temp))))

temp=[temp3 for n in range(rep)]
X_train_padded_new=np.vstack((X_train_padded,np.array(np.vstack(temp))))

In [96]:
from keras.layers.wrappers import Bidirectional

In [251]:
model=Sequential()

model.add(Bidirectional(LSTM(output_dim=32,return_sequences=True,activation='sigmoid',inner_activation='hard_sigmoid'),input_shape=(20, 154)))
model.add(Dropout(0.5))

model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(16))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(16))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(8))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(len(np.unique(sub_strings_label))+1)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adagrad',metrics=['accuracy','fmeasure'])  

In [ ]:
model.fit(X_train_padded_new, y_train_padded_new, batch_size=1, nb_epoch=20)

Epoch 1/20
38655/38655 [==============================] - 544s - loss: 0.0082 - acc: 0.9981 - fmeasure: 0.9970   
Epoch 2/20
38655/38655 [==============================] - 538s - loss: 0.0076 - acc: 0.9981 - fmeasure: 0.9970   
Epoch 3/20
38655/38655 [==============================] - 537s - loss: 0.0071 - acc: 0.9981 - fmeasure: 0.9971   
Epoch 4/20
38655/38655 [==============================] - 539s - loss: 0.0067 - acc: 0.9981 - fmeasure: 0.9971   
Epoch 5/20
38655/38655 [==============================] - 663s - loss: 0.0064 - acc: 0.9982 - fmeasure: 0.9971   
Epoch 6/20
38655/38655 [==============================] - 659s - loss: 0.0061 - acc: 0.9983 - fmeasure: 0.9972   
Epoch 7/20
38655/38655 [==============================] - 673s - loss: 0.0058 - acc: 0.9982 - fmeasure: 0.9972   
Epoch 8/20
38655/38655 [==============================] - 653s - loss: 0.0057 - acc: 0.9983 - fmeasure: 0.9973   
Epoch 9/20
38655/38655 [==============================] - 614s - loss: 0.0055 - acc: 0.9

In [ ]:
# for i in range(10):
#     X_valid=[sequence.pad_sequences(all_file_features[i],maxlen=20,dtype='float32',padding='post',truncating='post') ]
#     X_valid_padded=np.vstack(X_valid)

#     X_valid_tokens=[sequence.pad_sequences(all_file_tokens[i],maxlen=20,dtype='str',padding='post',truncating='post')]
#     X_valid_tokens_padded=np.vstack(X_valid_tokens)

#     y_valid=[sequence.pad_sequences(all_file_labels_array[i],maxlen=20,dtype='float32',padding='post',truncating='post')]
#     y_valid_padded=np.vstack(y_valid)

#     valid_prediction=model.predict(X_valid_padded,batch_size=100)
#     valid_prediction_classes=model.predict_classes(X_valid_padded,batch_size=100)

#     print(X_valid_tokens_padded[valid_prediction_classes!=0])

In [ ]:
actual=X_valid_tokens_padded[y_valid_padded[:,:,0]!=1][X_valid_tokens_padded[y_valid_padded[:,:,0]!=1]!="0.0"]

In [ ]:
X_valid=[sequence.pad_sequences(all_file_features[i],maxlen=20,dtype='float32',padding='post',truncating='post') for i in range(100,len(training_data))]
X_valid_padded=np.vstack(X_valid)

X_valid_tokens=[sequence.pad_sequences(all_file_tokens[i],maxlen=20,dtype='str',padding='post',truncating='post') for i in range(100,len(training_data))]
X_valid_tokens_padded=np.vstack(X_valid_tokens)

y_valid=[sequence.pad_sequences(all_file_labels_array[i],maxlen=20,dtype='float32',padding='post',truncating='post') for i in range(100,len(training_data))]
y_valid_padded=np.vstack(y_valid)

valid_prediction=model.predict(X_valid_padded,batch_size=100)
valid_prediction_classes=model.predict_classes(X_valid_padded,batch_size=100)

result=X_valid_tokens_padded[valid_prediction_classes!=0]

In [ ]:
actual_copy=actual.copy()
result_copy=result.copy()

In [ ]:
actual_list=list(actual)
actual_copy_list=list(actual_copy)
result_list=list(result)
result_copy_list=list(result_copy)

In [ ]:
for item in result_list:
    if item in actual_list:
        actual_list.remove(item)

In [ ]:
recall=1-(len(actual_list)/len(actual_copy_list))

In [ ]:
for item in actual_copy_list:
    if item in result_list:
        result_list.remove(item)

In [ ]:
precision=1-(len(result_list)/len(result_copy_list))

In [ ]:
print(precision,recall)